<a id="toc"></a>
# DEIMOS beamline simulation with PyOptiX
***

Contents :
1. [Optical elements declaration](#def_opt)
1. [Definition of optical parameters](#def_param)
1. [Alignment scripts](#alignement)
1. [Simulation execution](#exec)
1. [Visualisation](#visu)
1. [Monochromator resolution](#monores)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '21/OCT/2024'
__changed__ = '21/OCT/2024'


import ctypes

import numpy as np
import pandas as pd
import pyoptix
import pyoptix.classes as opx
from barc4xoc.aux_pyoptix import read_pyoptix_beam_from_csv, save_beam_data_to_csv
from barc4xoc.misc import coddington_equations
from scipy.constants import c, degree, eV, h, micro, milli, nano, pi, pico

hc = h*c/eV

pyoptix.set_aperture_active(False)
pyoptix.output_notebook()


<a id="def_opt"></a>
## Optical elements declaration
[Back to the top](#toc)

In [2]:
Deimos = opx.Beamline(name="Deimos - current")

# ------------------
# source
# ------------------
undulator_U52 = opx.UndulatorSource(name="U52")         # HU52 - 350eV - 2500eV
undulator_U65 = opx.UndulatorSource(name="U65")         # EMPHU65 - same energy range, but 1.8m downstream
undulator_SII = opx.UndulatorSource(name="SII")         # HU52 - 350eV - 2500eV with SOLEIL-II magnetic lattice

# ------------------
# optical elements
# ------------------
pupil = opx.PlaneFilm(name="pupil")

# M1
m1a = opx.PlaneMirror(name="M1A")
m1b = opx.ToroidalMirror(name="M1B")
m1c = opx.ToroidalMirror(name="M1C")

# monochromator
grating_1600 = opx.PlaneHoloGrating(name="grating_1600")   # VGD
grating_2400 = opx.PlaneGrating(name="grating_2400")       # MCA

m2 = opx.PlaneMirror(name="M2")
m3 = opx.CylindricalMirror(name="M3")
foc_hor = opx.PlaneFilm(name="foc_hor")
mono_exit_slit = opx.PlaneFilm(name="mono_exit_slit")

# M4
# m4a = opx.ToroidalMirror(name="M4A")
# m4b = opx.ToroidalMirror(name="M4B")
m4a = opx.ConicCylindricalMirror(name="M4A")
m4b = opx.ConicCylindricalMirror(name="M4B")

# M5
# m5a = opx.ToroidalMirror(name="M5A")
# m5b = opx.ToroidalMirror(name="M5B")
m5a = opx.ConicCylindricalMirror(name="M5A")
m5b = opx.ConicCylindricalMirror(name="M5B")

# ------------------
# endstations
# ------------------
cromag = opx.PlaneFilm(name="cromag")
mk2t = opx.PlaneFilm(name="mk2t")

In [3]:
undulators = [undulator_U52, undulator_U65, undulator_SII]
m1 = [m1b, m1c]
gratings = [grating_1600, grating_2400]
wolters = [[m4a, m5a], [m4b, m5b], []]
endstations = [cromag, mk2t]

for undulator in undulators:
    for m1bis in m1:
        for grating in gratings:
            beamline = [undulator, pupil, m1a, m1bis, grating, m2, m3, foc_hor, mono_exit_slit]           
            for focusing in wolters:
                if len(focusing) > 1:
                    chain_name = f"{undulator.name}_{m1bis.name}_G{grating.name.split('_')[1]}_WLT_{focusing[-2].name[-1]}"
                    Deimos.chains[chain_name] = beamline + focusing + endstations
                else:
                    chain_name = f"{undulator.name}_{m1bis.name}_G{grating.name.split('_')[1]}_mono_exit_slit"
                    Deimos.chains[chain_name] = beamline


In [ ]:
print("Low energy configurations:\n")
for chain_name in Deimos.chains:    
    if "M1B" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

In [ ]:
print("High energy configurations:\n")
for chain_name in Deimos.chains:    
    if "M1C" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

<a id="def_param"></a>
## Definition of optical parameters

In this section we define the static parameters of the optical elements. Characteristics that change with 
either configuration or energy are defined in the [Alignment scripts](#alignement) section.

[Back to the top](#toc)

In [ ]:
electron_parameters = pd.read_csv(r"./resources/2023-04-26-Dimensions-SOLEIL-actuel-pour-Stephane-BAC.csv", delimiter=";")
electron_parameters.loc[electron_parameters['BeamLineName'].str.contains('DEIMOS')]

### HU52 undulator

In [ ]:
src_name = "DEIMOS_1" # centre of straight section

e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["betaX(m)"].iloc[0]), 
                  eta_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etax(m)"].iloc[0]), 
                  etap_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etaxp(-)"].iloc[0]), 
                  alpha_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["alphax(-)"].iloc[0]),
                  beta_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["betaZ(m)"].iloc[0]), 
                  eta_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etaz(m)"].iloc[0]), 
                  etap_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etazp(-)"].iloc[0]), 
                  alpha_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["alphaz(-)"].iloc[0]))
e_beam.print_rms()

hu52 = opx.MagneticStructure(period_length=52.4e-3, number_of_periods=30)

undulator_U52.electron_beam = e_beam
undulator_U52.magnetic_structure = hu52

undulator_U52.write_syned_config(r".\resources\oasys_soleil_hu52","Soleil - HU52")

### HU65 undulator

In [ ]:
src_name = "DEIMOS_2" # centre of straight section

e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["betaX(m)"].iloc[0]), 
                  eta_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etax(m)"].iloc[0]), 
                  etap_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etaxp(-)"].iloc[0]), 
                  alpha_x=float(electron_parameters[electron_parameters.BeamLineName == src_name]["alphax(-)"].iloc[0]),
                  beta_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["betaZ(m)"].iloc[0]), 
                  eta_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etaz(m)"].iloc[0]), 
                  etap_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["etazp(-)"].iloc[0]), 
                  alpha_y=float(electron_parameters[electron_parameters.BeamLineName == src_name]["alphaz(-)"].iloc[0]))
e_beam.print_rms()

emphu65 = opx.MagneticStructure(period_length=65e-3, number_of_periods=24)

undulator_U65.electron_beam = e_beam
undulator_U65.magnetic_structure = emphu65

undulator_U65.write_syned_config(r".\resources\oasys_soleil_emphu65","Soleil - EMPHU65")

### SOLEIL-II lattice (HU52 undulator)

In [ ]:
e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.0906/100, current=0.500,
                  emittance_x=84.55*pico, emittance_y=25.36*pico,
                  beta_x=1.59, eta_x=0, etap_x=0, alpha_x=0,
                  beta_y=1.51, eta_y=0, etap_y=0, alpha_y=0)
e_beam.print_rms()

hu52 = opx.MagneticStructure(period_length=52.4e-3, number_of_periods=30)

undulator_SII.electron_beam = e_beam
undulator_SII.magnetic_structure = hu52

undulator_SII.write_syned_config(r".\resources\oasys_soleil-II_hu52","Soleil-II - HU52")

### Entrance pupil
[Back to the top](#toc)

In [10]:
pupil.distance_from_previous = 21
pupil.recording_mode = opx.RecordingMode.recording_output
pupil.next = m1a

### M1

[Back to the top](#toc)

In [ ]:
m1_grazing_angles = [1.5*degree, 0.43*degree]

m1_distances = opx.M1_triad_distances(incident_angle=m1_grazing_angles,
                                      transverse_distance=41e-3, 
                                      verbose=1)

#### - M1A

In [12]:
m1a.distance_from_previous = 0
m1a.phi = -90*degree # rad
m1a.recording_mode = opx.RecordingMode.recording_output

#### - M1B

In [ ]:
p = pupil.distance_from_previous + m1_distances[0][0]
qminor = 1E23
qmajor = (m1_distances[1][1]-m1_distances[0][1]) + 5 + 0.7 + 4 # we focus on the mochromator slit

rminor = coddington_equations(m1_grazing_angles[0]/degree, p=p, q=qminor)["R"][1]
rmajor = coddington_equations(m1_grazing_angles[0]/degree, p=p, q=qmajor)["R"][0]

print("Toroidal mirror with side-bounce reflection:")
print(f"- source at {p:.3f} m")
print(f"- (sagittal) image at {qminor:.2e} m")
print(f"- (meridional/tangential) image at {qmajor:.3f} m")

print("\nApplying the Coddington equations:")
print(f"- minor (sagittal) radius {rminor:.3f} m")
print(f"- major (meridional/tangential) radius {rmajor:.3f} m")

m1b.distance_from_previous = m1_distances[0][0]
m1b.theta = 2.53*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/rminor # m-1   # m1b.minor_curvature = 1/1.894 # m-1
m1b.major_curvature = 1/rmajor # m-1   # m1b.major_curvature = 1/228.0 # m-1
m1b.recording_mode = opx.RecordingMode.recording_output
m1b.next = grating_1600

#### - M1C

In [ ]:
p = pupil.distance_from_previous + m1_distances[1][0]
qminor = 1E23
qmajor = 5 + 0.7 + 4 # we focus on the mochromator slit

rminor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qminor)["R"][1]
rmajor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qmajor)["R"][0]

print("Toroidal mirror with side-bounce reflection:")
print(f"- source at {p:.3e} m")
print(f"- (sagittal) image at {qminor:.3e} m")
print(f"- (meridional/tangential) image at {qmajor:.3e} m")

print("\nApplying the Coddington equations:")
print(f"- minor (sagittal) radius {rminor:.3e} m")
print(f"- major (meridional/tangential) radius {rmajor:.3e} m")

m1c.distance_from_previous = m1_distances[1][0]
m1c.theta = 1.2*degree # rad
m1c.phi = 180*degree # rad
m1c.minor_curvature = 1/rminor   # m-1   # m1c.minor_curvature = 1/0.91  # m-1
m1c.major_curvature = 1/rmajor   # m-1   # m1c.major_curvature = 1/455   # m-1
m1b.recording_mode = opx.RecordingMode.recording_output
m1b.next = grating_2400

### Monochromator

[Back to the top](#toc)

#### Grating 1600 l/mm

In [ ]:
grating_1600.distance_from_previous = 5.0
grating_1600.line_density = 1600/milli
grating_1600.from_solemio(1e-10, 1e-10, 0.7660444, -0.56176)     # RC20240502 - where did you get this?
grating_1600.order_align = 1 
grating_1600.order_use = 1 
grating_1600.phi = -90*degree
grating_1600.recording_mode = opx.RecordingMode.recording_output
grating_1600.next = m2

grating_1600.show_vls_law(300e-3,3)

#### Grating 2400 l/mm

In [16]:
grating_2400.distance_from_previous = 5.0
grating_2400.line_density = 2400/milli
grating_2400.phi = -90*degree
grating_2400.order_align = 1
grating_2400.order_use = 1
grating_2400.recording_mode = opx.RecordingMode.recording_input
grating_2400.next = m2

#### M2

In [17]:
m2.phi = 180*degree
m2.recording_mode = opx.RecordingMode.recording_output
m2.next = m3

#### M3

In [ ]:
p = 1E23
qminor = 4
qmajor = 1E23

rminor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qminor)["R"][1]
rmajor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qmajor)["R"][0]

print("Toroidal mirror with side-bounce reflection:")
print(f"- source at {p:.3e} m")
print(f"- (sagittal) image at {qminor:.3e} m")
print(f"- (meridional/tangential) image at {qmajor:.3e} m")

print("\nApplying the Coddington equations:")
print(f"- minor (sagittal) radius {rminor:.3e} m")
print(f"- major (meridional/tangential) radius {rmajor:.3e} m")


m3.theta = m1_grazing_angles[1] # rad
m3.phi = 90*degree    # rad
m3.curvature = 1/rminor  # m-1
m3.axis_angle = 90*degree # sagittal cylinder 
m3.recording_mode = opx.RecordingMode.recording_output
m3.next = foc_hor

#### Horizontal focusing

In [29]:
foc_hor.distance_from_previous = 0.2
foc_hor.phi = 90*degree # rad
foc_hor.recording_mode = opx.RecordingMode.recording_output
foc_hor.next = mono_exit_slit

### Mono exit slits

In [30]:
mono_exit_slit.distance_from_previous = 3.8
mono_exit_slit.phi = 0*degree # rad
# mono_exit_slit.add_rectangular_stop(20*micro, 20*micro, opacity=0)
mono_exit_slit.recording_mode = opx.RecordingMode.recording_output

### Focusing optics

[Back to the top](#toc)

#### M4A-E

In [21]:
m4a.distance_from_previous = 5.5
m4a.theta = 1.2*degree
m4a.phi = 90*degree
m4a.minor_curvature = 1/188.1 # m-1
m4a.major_curvature = 1/296.2 # m-1
m4a.recording_mode = opx.RecordingMode.recording_output
m4a.next = m5a 

#### M4B-E

In [22]:
m4b.distance_from_previous = 5.5
m4b.theta = 1.2*degree
m4b.phi = 90*degree
m4b.minor_curvature = 1/0.3761 # m-1
m4b.major_curvature = 1/347.3 # m-1
m5a.recording_mode = opx.RecordingMode.recording_input
m4b.next = m5b 

#### M5A-E

In [23]:
m5a.distance_from_previous = 0.4
m5a.theta = 1.2*degree
m5a.minor_curvature = 1/0.0833 # m-1
m5a.major_curvature = 1/967. # m-1
m5a.recording_mode = opx.RecordingMode.recording_input
m5a.next = cromag 

### M5B-E

In [24]:
m5b.distance_from_previous = 0.4
m5b.theta = 1.2*degree
m5b.minor_curvature = 1/0.15   # m-1
m5b.major_curvature = 1/3000.0 # m-1
m5b.recording_mode = opx.RecordingMode.recording_input
m5b.next = cromag 

### Endstations

[Back to the top](#toc)

#### CroMag

In [25]:
cromag.distance_from_previous = 3 # m
cromag.phi = -90*degree
cromag.recording_mode = opx.RecordingMode.recording_output
cromag.next = mk2t

#### mK2T

In [26]:
mk2t.distance_from_previous = 1.76
mk2t.phi = -90*degree
mk2t.recording_mode = opx.RecordingMode.recording_output

<a id="alignement"></a>
## Alignment scripts

In this section we define the all the parameters that change with either configuration or energy. These scripts are tailored
for each 

[Back to the top](#toc)

### Undulators

In [27]:
def align_undulator(active_chain, wavelength, **kwargs):
    """
    Sets photon beam size and divergence as a function of wavelength in [m]
    """

    verbose = kwargs.get("verbose", False)
    dist_slit = kwargs.get("dist_slit", 21)
    
    if verbose:
        print("\n>>>> Aligning undulator")

    active_chain[0].set_undulator(wavelength, **kwargs)
    active_chain[1].distance_from_previous = dist_slit
    if "65" in active_chain[0].name:
        active_chain[1].distance_from_previous -= (
            float(electron_parameters[electron_parameters.BeamLineName == "DEIMOS_2"]["Long_Pos(m)"].iloc[0])-
            float(electron_parameters[electron_parameters.BeamLineName == "DEIMOS_1"]["Long_Pos(m)"].iloc[0])
        )


### M1 

In [28]:
def align_m1(active_chain, m1_distances, **kwargs):
    """
    Sets the M1A angle and distance from mono first optical element to M1bis
    """
    
    dist_next = kwargs.get("dist_next", 5)
    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning M1")
   
    active_chain[2].theta = active_chain[3].theta
    active_chain[2].next  = active_chain[3]
    active_chain[3].next = active_chain[4]
    active_chain[4].distance_from_previous = dist_next

    if "1B" in active_chain[3].name:
        active_chain[4].distance_from_previous += (m1_distances[1][1]-m1_distances[0][1])       

    if verbose:
        name = active_chain[3].name
        angle = active_chain[3].theta/degree
        dist = active_chain[3].distance_from_previous
        print(f"M1A-{name} alignment: grazing angle {angle:.2f} degrees and {dist:.3f} m between them")

### Monochromator 

In [29]:
def align_mono(active_chain, wavelength, alignment_condition, alignment_condition_value,
               GM2_trans_dist, GM3_proj_dist, **kwargs):
    """
    Sets the grating and M2 angles and relative distances between G/M2/M3
    """

    verbose = kwargs.get("verbose", False)

    grating = None
    for count, oe in enumerate(active_chain):
        if "reseau" in oe.name.lower() or "grating" in oe.name.lower():
            grating = oe
            mono_oe_pos = count

    if grating is None:
        raise ValueError("No grating appears in this beamline configuration")
    
    gdict = opx.align_grating(grating, verbose=0, 
                             apply_alignment=True, 
                             return_parameters=True,
                             condition=alignment_condition, 
                             condition_value=alignment_condition_value, 
                             lambda_align=wavelength, 
                             order=grating.order_align,
                             line_density=grating.line_density)
    
    gm2 = GM2_trans_dist/np.sin(gdict["deviation"])
    m2m3 = GM3_proj_dist-(GM2_trans_dist/np.tan(gdict["deviation"]))

    active_chain[int(mono_oe_pos+1)].theta = gdict["deviation"]/2
    active_chain[int(mono_oe_pos+1)].distance_from_previous = gm2 

    active_chain[int(mono_oe_pos+2)].distance_from_previous = m2m3

    if verbose:
        print(f"\n>>>> {grating.name} grating alignment for a "+
              f"{alignment_condition} {alignment_condition_value} - wavelength " +
              f"{wavelength/nano:.3f} nm (E={hc/wavelength:.3f} eV)")
        print(f"> alpha {gdict['alpha_deg']:.6f} deg")
        print(f"> beta {gdict['beta_deg']:.6f} deg")
        print(f"> G-M2 distance {gm2:.6f} m")
        print(f"> theta_m2 {(gdict['alpha_deg']+gdict['beta_deg'])/2:.6f} deg")
        print(f"> M2-M3 distance {m2m3:.6f} m")


### Beamline alignment procedure

In [30]:
def align_call(alignment_wvl, emission_wvl, alignment_condition, alignment_condition_value, 
               m1_distances, GM2_trans_dist, GM3_proj_dist, **kwargs):

    align_undulator(Deimos.active_chain, emission_wvl, **kwargs)
    align_m1(Deimos.active_chain, m1_distances, **kwargs)
    align_mono(Deimos.active_chain, alignment_wvl, alignment_condition, 
               alignment_condition_value, GM2_trans_dist, GM3_proj_dist, **kwargs)
    
Deimos.align_steps = align_call

def set_bl(energy_alignment, alignment_condition, alignment_condition_value, 
               m1_distances=m1_distances, GM2_trans_dist=20e-3, GM3_proj_dist=700e-3, **kwargs):

    energy_radiate =  kwargs.get("energy_radiate", energy_alignment)
    dE =  kwargs.get("dE", 0)
    rays = kwargs.get("rays", 500)

    alignment_wvl = hc/energy_alignment
    emission_wvl = hc/energy_radiate

    Deimos.align(alignment_wvl, emission_wvl, 
                 alignment_condition=alignment_condition, 
                 alignment_condition_value=alignment_condition_value, 
                 m1_distances=m1_distances, GM2_trans_dist=GM2_trans_dist,
                 GM3_proj_dist=GM3_proj_dist, **kwargs)

    Deimos.clear_impacts(clear_source=True)
    Deimos.active_chain[0].nrays = rays
    if dE==0:
        Deimos.generate(emission_wvl)
    else:
        for E in np.arange(energy_radiate*(1-dE), energy_radiate*(1+dE)):
            Deimos.generate(hc/E)
    Deimos.radiate()

<a id="exec"></a>
## Simulation execution
[Back to the top](#toc)

### Available beamline configurations

In [ ]:
print("Low energy configurations:")
for chain_name in Deimos.chains:    
    if "1600" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

In [ ]:
print("High energy configurations:")
for chain_name in Deimos.chains:    
    if "2400" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

The beamline can be conviniently simulated by selecting an **active chain** and an energy:

```python
Deimos.active_chain = "U52_G1600_mono_exit_slit"
set_bl(350, alignment_condition="cff", alignment_condition_value=0.2, verbose=True)
```

To check the orientation of the optical elements, draw the beamline:

```python
Deimos.show_active_chain_orientation()
Deimos.draw_active_chain()
spots = Deimos.draw_to_scale()
```

<a id="visu"></a>
# Visualisations 
[Back to the top](#toc)

In [33]:
rays = 5000
E = 350

### HU52 with 1600 l/mm grating - E=350 eV

[Back to the top](#toc)

In [ ]:
Deimos.active_chain = "U52_G1600_mono_exit_slit"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=rays, center_undulator=0)

#### Horizontal focusing slits

In [ ]:
oe = foc_hor
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False,) 

#### Mono exit slit

In [ ]:
oe = mono_exit_slit
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

### Endstations with M4A-M5A (Wolter A) pair

In [ ]:
Deimos.active_chain = "U52_G1600_WLT_A"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=rays, center_undulator=0)

#### cromag

In [ ]:
oe = cromag
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

#### mK2T

In [ ]:
oe = mk2t
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:

opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

#### Endstations with M4B-M5B pair

In [ ]:
Deimos.active_chain = "U52_G1600_WLT_B"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=rays, center_undulator=0)

#### cromag

In [ ]:
oe = cromag
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

#### mK2T

In [ ]:
oe = mk2t
f = opx.get_optimal_focalization_distance(oe, verbose=True)

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:

opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

<a id="monores"></a>
## Monochromator resolution 
[Back to the top](#toc)